In [1]:
pip install torch torchvision opencv-python pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import torch
from torchvision import models, transforms
from PIL import Image

In [3]:
model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to C:\Users\Shekhani Laptops/.cache\torch\hub\checkpoints\deeplabv3_resnet101_coco-586e9e4e.pth
100%|████████████████████████████████████████████████████████████████████████████████| 233M/233M [23:12<00:00, 176kB/s]


In [4]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
def remove_background(image_path):
    # Load the image
    input_image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)

    # Move to GPU if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    # Perform inference
    with torch.no_grad():
        output = model(input_batch)['out'][0]
    
    # Get the mask (Class 0 is background, Class 15 is person)
    output_predictions = output.argmax(0).byte().cpu().numpy()
    
    # Convert the mask into a 3-channel image
    mask = output_predictions == 15  # Assuming class 15 represents the person
    mask = np.stack([mask]*3, axis=-1)
    
    # Convert the PIL image to a NumPy array
    image_np = np.array(input_image)
    
    # Apply the mask to remove the background
    result_image = image_np * mask
    
    # Convert the result to an image and save it
    result = Image.fromarray(result_image.astype(np.uint8))
    result.show()  # Display the image
    result.save('output.png')  # Save the output

In [6]:
image_path = 'test.jpeg'
remove_background(image_path)

In [7]:
image_path = '1.jpeg'
remove_background(image_path)